In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import glob

import matplotlib as mpl

import matplotlib.pyplot as plt

import math as mt

import cartopy.crs as ccrs

import cartopy.feature as cfeature

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

#AO Calculation

In [4]:
# Calculate AO - input std normal latitude wighted 1000mb z
ZData = xr.open_dataset('../../../Data/CMIP6/GFDL-ESM4/NDJF/weighted1000Std.nc')
#SLPData = xr.open_dataset('../eraDown/ERA5_2degree_Down/DailyMean/weighted1000.nc')

ZData.zg.values

ZData.zg.values=xr.where(np.isnan(ZData.zg.values),  0.000000000001,ZData.zg.values)
ZData.zg.values

array([[[ 1.0000000e-12,  1.0000000e-12,  1.0000000e-12, ...,
          1.0000000e-12,  1.0000000e-12,  1.0000000e-12],
        [ 1.0000000e-12,  1.0000000e-12,  1.0000000e-12, ...,
          1.0000000e-12,  1.0000000e-12,  1.0000000e-12],
        [ 1.0000000e-12,  1.0000000e-12,  1.0000000e-12, ...,
          1.0000000e-12,  1.0000000e-12,  1.0000000e-12],
        ...,
        [-2.4311945e-01, -2.3561670e-01, -2.2831504e-01, ...,
         -2.6948228e-01, -2.6221198e-01, -2.5467190e-01],
        [-2.2307622e-01, -2.1974581e-01, -2.1421339e-01, ...,
         -2.3277186e-01, -2.2959420e-01, -2.2636069e-01],
        [-1.4281146e-01, -1.4220323e-01, -1.4159340e-01, ...,
         -1.4461939e-01, -1.4402023e-01, -1.4341736e-01]],

       [[ 1.0000000e-12,  1.0000000e-12,  1.0000000e-12, ...,
          1.0000000e-12,  1.0000000e-12,  1.0000000e-12],
        [ 1.0000000e-12,  1.0000000e-12,  1.0000000e-12, ...,
          1.0000000e-12,  1.0000000e-12,  1.0000000e-12],
        [ 1.0000000e-12, 

In [5]:


t1=ZData.zg.stack(z=("lat", "lon"))
# fit scaler on training data
zg_PC =PCA().fit_transform(t1)


In [6]:
print(zg_PC[:,0].shape)
AO_ds = xr.Dataset({'AO': (('time'), zg_PC[:,0])}, coords={'time': ZData.time})


norm = StandardScaler().fit(zg_PC[:,0].reshape(-1, 1))
    # transform training data
aoX = norm.transform(zg_PC[:,0].reshape(-1, 1))
aoX.shape

AO_ds = xr.Dataset({'AO': (('time'), aoX[:,0]*-1.0)}, coords={'time': ZData.time})


(19800,)


In [10]:
AO_ds.to_netcdf('GFDL-CM4-AOindex-NDJF-Daily-1980-2014.nc')

In [8]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.max(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(2.845212, dtype=float32)
Coordinates:
    time     object 1930-12-28 12:00:00

In [ ]:
#AO_ds.AO.where(AO_ds.AO>2.0, drop=True).squeeze()

In [9]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.min(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(-4.295782, dtype=float32)
Coordinates:
    time     object 1965-02-17 12:00:00

In [ ]:
#AO_ds.AO.where(AO_ds.AO<-2, drop=True).squeeze()

#EU Calculation

In [11]:
zLevData = xr.open_dataset('../../../Data/CMIP6/GFDL-ESM4/NDJF/zgNH_NDJFday_GFDL-CM4_historical_r1i1p1f1_gr2_18500101-20141231.nc')

In [12]:
z500=zLevData.zg.sel(plev=50000)

In [13]:
z145E40N=z500.sel(lat=40,lon=145,method='nearest')

In [14]:
#z145E40N

In [15]:
z20E55N=z500.sel(lat=55,lon=20,method='nearest')

In [16]:
#z20E55N

In [17]:
z75E55N=z500.sel(lat=55,lon=75,method='nearest')

In [18]:
#z75E55N

In [19]:
#Calculate Anomaly
z145E40N_Anom = z145E40N.groupby("time.dayofyear") - z145E40N.groupby("time.dayofyear").mean("time")
z75E55N_Anom  = z75E55N.groupby("time.dayofyear")  - z75E55N.groupby("time.dayofyear").mean("time")
z20E55N_Anom  = z20E55N.groupby("time.dayofyear")  - z20E55N.groupby("time.dayofyear").mean("time")

In [20]:
# calculate std normal anomalies
z145E40N_AnomStd = z145E40N_Anom.groupby("time.dayofyear")  / z145E40N.groupby("time.dayofyear").std("time")
z75E55N_AnomStd  = z75E55N_Anom.groupby("time.dayofyear")   / z75E55N.groupby("time.dayofyear").std("time")
z20E55N_AnomStd  = z20E55N_Anom.groupby("time.dayofyear")   / z20E55N.groupby("time.dayofyear").std("time")

In [21]:
EUVal = -(1.0/4.0)*z20E55N_AnomStd + (1.0/2.0)*z75E55N_AnomStd - (1.0/4.0)*z145E40N_AnomStd

In [22]:
EUVal

<xarray.DataArray 'zg' (time: 19800)>
array([-0.47518525, -0.56123552, -0.96478767, ..., -0.20147391,
       -0.00195609, -0.24391161])
Coordinates:
  * time       (time) object 1850-01-01 12:00:00 ... 2014-12-31 12:00:00
    plev       float64 5e+04
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365
    lon        float64 143.8

In [23]:
EU_ds = xr.Dataset({'EU': (('time'), EUVal)}, coords={'time': zLevData.time})

In [24]:
EU_ds.to_netcdf('GFDL-CM4-EUindex-NDJF-Daily-1980-2014.nc')